# 컨브넷(convnet)이 뭐냐?

### 합성곱 신경망(convolutional neural network)

### MNIST 숫자 이미지 분류에 사용해보자.

In [1]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

### 간단한 컨브넷 만들기

In [4]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


In [5]:
model.summary() # 컨브넷 구조 출력

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


### Conv2D와 MaxPooling2D 층의 출력은 (height, width, channels)크기의 3D텐서이다. 높이와 너비 차원은 네트워크가 깊어질수록 작아지는 경향이 있다. 채널의 수는 Conv2D층에 전달된 첫번째 매개변수에 의해 조절된다.

다음 단계에서 마지막 층의 ((3, 3, 64) 크기인) 출력 텐서를 연결 네트워크에 주입한다. 이 분류기는 1D벡터를 처리하는데, 이전 층의 출력이 3D텐서이다. 그래서 출력을 1D텐서로 펼쳐야한다.

### 컨브넷 위에 분류기 추가하기

In [7]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)              

(3, 3, 64)의 출력이 (576, )크기의 벡터로 Dence층에 주입되는 것을 볼수있다!!

### MNIST 이미지에 컨브넷 훈련하기

In [11]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 28s 30ms/step - loss: 0.1644 - accuracy: 0.9481
Epoch 2/5
938/938 [==============================] - 30s 32ms/step - loss: 0.0464 - accuracy: 0.9860
Epoch 3/5
938/938 [==============================] - 30s 31ms/step - loss: 0.0326 - accuracy: 0.9900
Epoch 4/5
938/938 [==============================] - 30s 32ms/step - loss: 0.0248 - accuracy: 0.9923
Epoch 5/5
938/938 [==============================] - 30s 32ms/step - loss: 0.0193 - accuracy: 0.9940


In [14]:
test_loss,  test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 2s 5ms/step - loss: 0.0277 - accuracy: 0.9910


In [16]:
test_loss

0.027740478515625

In [17]:
test_acc

0.9909999966621399